### Implementing a ChatGPT App with LangChain from Scratch

In [3]:
from dotenv import load_dotenv, find_dotenv # type: ignore
load_dotenv(find_dotenv(), override=True) 

from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import SystemMessage # type: ignore # It sets the behavior of the chatbot.
from langchain.chains import LLMChain # type: ignore                                                                                
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate # type: ignore

# Creating ChatOpenAI instance
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# Creating ChatPromptTemplate instance
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# Creating LLMChain instance and starting the conversation
chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

# Starting the conversation by running the chain
while True:
    
    # Get User Input
    userInput = input('Your prompt: ')

    # Check if User wants to Quit the conversation
    if userInput.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    # Invoke the chain
    response = chain.invoke({'content': userInput})
    print(response)
    print('-' * 50)
    

{'content': 'Who is the PM of India?', 'text': 'As of now, the Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister since May 2014.'}
--------------------------------------------------
Goodbye!


### Adding Chat Memory Using ConversationBufferMemory

Memory is used to store information about past interactions with the LLM

In [9]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Imports
from langchain.memory import ConversationBufferMemory # It stores all the messages that the Human sends and all the responses that the Chatbot sends.

# MessagesPlaceholder: allows us to insert 'List of Historical messages' into the prompt
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder 

# Create ChatOpenAI instance
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Create memory 
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True) # chat_history: is the 'key' where the 'chat memory' will be stored.

# 3. Add MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored. The placeholder is set before the HumanMessagePromptTemplate
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# 4. Add the memory to the chain
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=False)

# Starting the conversation by running the chain
while True:
    
    # Get User Input
    userInput = input('Your prompt: ')

    # Check if User wants to Quit the conversation
    if userInput.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    # Invoke the chain
    response = chain.invoke({'content': userInput})
    #print(response)
    print(response['text'])
    print('-' * 50)
    

{'content': 'earth mass is', 'chat_history': [HumanMessage(content='earth mass is'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms. It is the third planet from the Sun in our solar system and is known for its diverse ecosystem and habitability for various forms of life.")], 'text': "The Earth's mass is approximately 5.97 x 10^24 kilograms. It is the third planet from the Sun in our solar system and is known for its diverse ecosystem and habitability for various forms of life."}
--------------------------------------------------
{'content': 'its diameter is', 'chat_history': [HumanMessage(content='earth mass is'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms. It is the third planet from the Sun in our solar system and is known for its diverse ecosystem and habitability for various forms of life."), HumanMessage(content='its diameter is'), AIMessage(content="The approximate diameter of Earth is 12,742 kilometers. This measureme

### Saving Chat Sessions

In [13]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
 
# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

# Create ChatOpenAI instance
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# Creating FileChatMessageHistory instance. All the questions and answers will be stored in 'chat_history.json' file
history = FileChatMessageHistory('chat_history.json')

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
# chat_history: is the 'key' where the 'chat memory' will be stored
# chat_memory: is the 'history' object that we created above
memory = ConversationBufferMemory(memory_key = 'chat_history', chat_memory = history, return_messages=True)

# Creating ChatPromptTemplate instance
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# Creating LLMChain instance and starting the conversation
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=False)

# Starting the conversation by running the chain
while True:
    
    # Get User Input
    userInput = input('Your prompt: ')

    # Check if User wants to Quit the conversation
    if userInput.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    # Invoke the chain
    response = chain.invoke({'content': userInput})
    print(response['text'])
    print('-' * 50)
    
    

The speed of light in a vacuum is approximately 299,792,458 meters per second or about 186,282 miles per second.
--------------------------------------------------
The speed of light in water is about 225,000 kilometers per second, which is slower than the speed of light in a vacuum. The speed of light in a medium like water is slower due to interactions with atoms and molecules in the material.
--------------------------------------------------
The speed of light in air is about 299,702 kilometers per second, which is slightly slower than the speed of light in a vacuum but faster than in water. Air has less influence on the speed of light compared to water due to its lower density and fewer interactions with light particles.
--------------------------------------------------
Goodbye!


In [14]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='light spped in vaccum?'), AIMessage(content='The speed of light in a vacuum is approximately \\(299,792,458\\) meters per second or about \\(186,282\\) miles per second.'), HumanMessage(content='light speed in vaccum?'), AIMessage(content='The speed of light in a vacuum is approximately 299,792,458 meters per second or about 186,282 miles per second.'), HumanMessage(content='Light speed in vaccum?'), AIMessage(content='The speed of light in a vacuum is approximately 299,792,458 meters per second or about 186,282 miles per second.'), HumanMessage(content='Light speed in vaccum?'), AIMessage(content='The speed of light in a vacuum is approximately 299,792,458 meters per second or about 186,282 miles per second.'), HumanMessage(content='its speed in water'), AIMessage(content='The speed of light in water is about 225,000 kilometers per second, which is slower than the speed of light in a vacuum. The speed of light in a medium like water is slower due to interactions